<a href="https://colab.research.google.com/github/DataFutebol/DataFut/blob/main/Alest2Ext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas networkx

In [ ]:
pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
df = pd.read_csv('dados.csv', delimiter = ';')
dados = df.rename(columns = {'-24.1036':'x',
                '-49.7900':'y'})
dados

,x,y
0,-19.5603,-46.9653
1,-9.8661,-56.1044
2,-22.6394,-50.4525
3,-10.2500,-59.3833
4,-10.9853,-37.0733
...,...,...
241,-23.0069,-47.1344
242,-14.8631,-40.8625
243,-20.2581,-40.2864
244,-20.4233,-49.9881


In [ ]:
import pandas as pd
import networkx as nx
from sklearn.neighbors import BallTree

# Extrai as coordenadas (assumindo colunas 'x' e 'y')
coordenadas = dados[['x', 'y']]

# Constrói a árvore para consulta rápida dos vizinhos mais próximos
tree = BallTree(coordenadas, leaf_size=15)

# Cria um grafo não dirigido valorado
grafo = nx.Graph()

# Adiciona os nós ao grafo
grafo.add_nodes_from(dados.index)

# Calcula a distância euclidiana entre dois pontos
def calcular_distancia(ponto1, ponto2):
    return ((ponto1[0] - ponto2[0])**2 + (ponto1[1] - ponto2[1])**2)**0.5

# Encontra os 3 vizinhos mais próximos para cada ponto e adiciona arestas ponderadas
for ponto in dados.index:
    ponto_coord = coordenadas.loc[ponto].values
    distancias, indices = tree.query([ponto_coord], k=4)

    for i, distancia in zip(indices[0][1:], distancias[0][1:]):
        vizinho = i
        peso = calcular_distancia(ponto_coord, coordenadas.loc[vizinho].values)
        grafo.add_edge(ponto, vizinho, weight=round(peso, 2))

# Função para encontrar a Árvore Geradora Mínima usando o algoritmo de Prim
def prim_mst(grafo):
    mst = nx.minimum_spanning_tree(grafo, algorithm='prim')
    return mst

# Adiciona a distância euclidiana como peso nas arestas do grafo
for aresta in grafo.edges(data=True):
    ponto1 = coordenadas.loc[aresta[0]].values
    ponto2 = coordenadas.loc[aresta[1]].values
    distancia = calcular_distancia(ponto1, ponto2)
    grafo[aresta[0]][aresta[1]]['weight'] = round(distancia, 2)

# Calcula a Árvore Geradora Mínima usando o algoritmo de Prim
mst = prim_mst(grafo)

# Salva as informações da MST em um arquivo de texto
output_filename = 'saida.txt'
with open(output_filename, 'w') as output_file:
    output_file.write("Número do vértice inicial - Número do vértice final - Pertence ao MST\n")
    for aresta in mst.edges(data=True):
        pertence_mst = 1 if 'weight' in aresta[2] else 0
        output_file.write(f"{aresta[0]} - {aresta[1]} - {pertence_mst}\n")

print(f"As informações da MST foram salvas em '{output_filename}'.")

As informações da MST foram salvas em 'saida.txt'.


In [ ]:
import pandas as pd
import networkx as nx
from sklearn.neighbors import BallTree

# Extrai as coordenadas (assumindo colunas 'x' e 'y')
coordenadas = dados[['x', 'y']]

# Constrói a árvore para consulta rápida dos vizinhos mais próximos
tree = BallTree(coordenadas, leaf_size=15)

# Cria um grafo não dirigido valorado
grafo = nx.Graph()

# Adiciona os nós ao grafo
grafo.add_nodes_from(dados.index)

# Calcula a distância euclidiana entre dois pontos
def calcular_distancia(ponto1, ponto2):
    return ((ponto1[0] - ponto2[0])**2 + (ponto1[1] - ponto2[1])**2)**0.5

# Encontra os 3 vizinhos mais próximos para cada ponto e adiciona arestas ponderadas
for ponto in dados.index:
    ponto_coord = coordenadas.loc[ponto].values
    distancias, indices = tree.query([ponto_coord], k=4)

    for i, distancia in zip(indices[0][1:], distancias[0][1:]):
        vizinho = i
        peso = calcular_distancia(ponto_coord, coordenadas.loc[vizinho].values)
        grafo.add_edge(ponto, vizinho, weight=round(peso, 2))

# Função para encontrar a Árvore Geradora Mínima usando o algoritmo de Prim
def prim_mst(grafo):
    mst = nx.minimum_spanning_tree(grafo, algorithm='prim')
    return mst

# Adiciona a distância euclidiana como peso nas arestas do grafo
for aresta in grafo.edges(data=True):
    ponto1 = coordenadas.loc[aresta[0]].values
    ponto2 = coordenadas.loc[aresta[1]].values
    distancia = calcular_distancia(ponto1, ponto2)
    grafo[aresta[0]][aresta[1]]['weight'] = round(distancia, 2)

# Calcula a Árvore Geradora Mínima usando o algoritmo de Prim
mst = prim_mst(grafo)

# Exibe as informações da MST
print("Número do vértice inicial - Número do vértice final - Pertence ao MST")
for aresta in mst.edges(data=True):
    print(f"{aresta[0]} -> {aresta[1]} -> 1")

Número do vértice inicial - Número do vértice final - Pertence ao MST
0 -> 173 -> 1
0 -> 234 -> 1
1 -> 117 -> 1
2 -> 134 -> 1
2 -> 47 -> 1
2 -> 152 -> 1
2 -> 175 -> 1
3 -> 109 -> 1
4 -> 130 -> 1
5 -> 192 -> 1
6 -> 85 -> 1
6 -> 26 -> 1
7 -> 8 -> 1
7 -> 226 -> 1
8 -> 121 -> 1
8 -> 133 -> 1
9 -> 195 -> 1
9 -> 23 -> 1
10 -> 116 -> 1
10 -> 11 -> 1
10 -> 21 -> 1
12 -> 124 -> 1
12 -> 244 -> 1
13 -> 179 -> 1
14 -> 40 -> 1
14 -> 49 -> 1
15 -> 213 -> 1
16 -> 187 -> 1
16 -> 115 -> 1
17 -> 95 -> 1
17 -> 28 -> 1
18 -> 200 -> 1
19 -> 31 -> 1
19 -> 210 -> 1
20 -> 204 -> 1
20 -> 159 -> 1
21 -> 206 -> 1
22 -> 158 -> 1
22 -> 30 -> 1
23 -> 187 -> 1
23 -> 184 -> 1
24 -> 119 -> 1
25 -> 164 -> 1
25 -> 128 -> 1
26 -> 137 -> 1
27 -> 49 -> 1
29 -> 156 -> 1
29 -> 109 -> 1
30 -> 237 -> 1
31 -> 233 -> 1
31 -> 129 -> 1
32 -> 167 -> 1
32 -> 51 -> 1
33 -> 41 -> 1
33 -> 131 -> 1
34 -> 231 -> 1
34 -> 68 -> 1
34 -> 88 -> 1
35 -> 57 -> 1
36 -> 131 -> 1
36 -> 39 -> 1
37 -> 133 -> 1
37 -> 236 -> 1
38 -> 118 -> 1
39 -> 83 